# Z3 SAT Constraints Encodings

In this notebook we are going to show several implementation in z3 of the cardinality constraints and show how they affect the efficiency of the solver. 

In [1]:
from z3 import *